In [12]:
import os, glob
import numpy as np
import pandas as pd
get_f_list= glob.glob("../result/merge_flower_price/*台北*")
pd.set_option('display.max_rows', 10)
flowers = []
for fn in get_f_list:
    flowers.append(fn.split("\\")[-1].split("_")[0])

flower = flowers[0]
fn_list = glob.glob("../result/merge_flower_price/*" + flower + "*.csv")
fn_list

['../result/merge_flower_price\\Anthurium_台中.csv',
 '../result/merge_flower_price\\Anthurium_台北.csv',
 '../result/merge_flower_price\\Anthurium_台南.csv',
 '../result/merge_flower_price\\Anthurium_彰化.csv',
 '../result/merge_flower_price\\Anthurium_高雄.csv']

In [37]:
def trans_by_week(fn):
    c = ["date", "market", "avg_price", "total_count", "year", "week"]
    df_tmp = pd.read_csv(fn, encoding="utf-8",names=c)
    # pd.set_option('display.max_columns', 10)
    # 將53周(一年最後一天/最後第二天歸在 52 周)
    df_tmp["week"] = df_tmp["week"].replace(53,52)
    df_tmp["total_value"] = df_tmp["avg_price"] * df_tmp["total_count"]
    market = df_tmp["market"][0]
    # 計算 每周總合
    df_result = df_tmp.groupby(["year", "week"]).agg({"total_value":["sum"],"total_count":"sum"})
    df_result["week_avg"]  =df_result[('total_value', 'sum')] / df_result[('total_count', 'sum')]
    df_result["week_sale"] = df_result[('total_count', 'sum')]
    df_result = df_result.drop(columns=[('total_value', 'sum'),('total_count', 'sum')])
    df_result["market"] = market
    df_result = df_result.reset_index()
    df_result = pd.DataFrame(np.array(df_result), columns=["year", "week", "w_avg", "w_sale", "market"])
    df_result["price_diff"]  = df["w_avg"].pct_change()[1:]
    df_result["y_w"] =df_result["year"].map(str) + "_" + df_result["week"].map(str)
    return df_result

In [40]:
flowers = []
for fn in get_f_list:
    flowers.append(fn.split("\\")[-1].split("_")[0])

# flower = flowers[0]
for flower in flowers:
    fn_list = glob.glob("../result/merge_flower_price/*" + flower + "*.csv")
    f = 0
    for fn in fn_list:
        df = trans_by_week(fn)
        if f ==0:
            df_all = df
            f = f + 1
        else:
            df_all = df_all.append(([df_all,df]))

    df_all = df_all.reset_index().drop(columns="index")
    df_all
    fn_save = "../result/flower_price_byweek/" + flower + "_pbyweek.csv"
    df_all.to_csv(fn_save, encoding="utf-8", index=False)
    print(flower, "saved")
# df.to_csv("test0515.csv", encoding="utf-8", index=False)

Anthurium saved
Chrysanthemum saved
Eustoma saved
OrientalLily saved
Rose saved
